# Data Extractor

Extracting researches, authors, and affiliations

In [13]:
from os import listdir
from os.path import isfile, join, abspath, dirname
import json
import pandas as pd
import numpy as np
from tqdm import tqdm

In [14]:
authors_id_set = set()
affiliations_id_set = set()

authors_list = []
affiliations_list = []

In [15]:
def extract_json_from_data(data):
    # Check if there is key in data
    assert "abstracts-retrieval-response" in data

    data = data["abstracts-retrieval-response"]

    if data.get("item").get("bibrecord").get("tail") is None:
        new_ref_id_list = []
    else:
        if type(data.get("item").get("bibrecord").get("tail").get("bibliography").get("reference")) is dict:
            ref_id_list = [data.get("item").get("bibrecord").get("tail").get("bibliography").get("reference").get("ref-info").get("refd-itemidlist").get("itemid")]
        else:
            ref_id_list = [field.get("ref-info").get("refd-itemidlist").get("itemid") for field in data.get("item").get("bibrecord").get("tail").get("bibliography").get("reference")] if data.get("item").get("bibrecord").get("tail") else []
        new_ref_id_list = []
        for r in ref_id_list:
            if type(r) is list:
                new_ref = [g.get("$") for g in r if g.get("@idtype") in "SGR"][0]
            else:
                new_ref = r.get("$")
            new_ref_id_list.append(new_ref)

    # Authors
    if data.get("authors").get("author"):
        for author in data.get("authors").get("author"):
            author_id = author.get("@auid")
            if author_id in authors_id_set:
                continue

            if type(author.get("affiliation")) is dict:
                aff_list = [author.get("affiliation")]
            else :
                aff_list = author.get("affiliation") if author.get("affiliation") else []

            authors_id_set.add(author_id)
            authors_list.append({
                "id": author_id,
                "given_name": author.get("ce:given-name"),
                "initials": author.get("ce:initials"),
                "surname": author.get("ce:surname"),
                "indexed_name": author.get("ce:indexed-name"),
                "affiliations_id": "|".join(set([a.get("@id") for a in aff_list]))
            })

    # Affiliations
    if data.get("affiliation"):
        if type(data.get("affiliation")) is dict:
            aff_list = [data.get("affiliation")]
        else:
            aff_list = data.get("affiliation") if data.get("affiliation") else []
            
        for aff in aff_list:
            
            if aff.get("@id") in affiliations_id_set:
                continue
            

            affiliations_id_set.add(aff.get("@id"))
            affiliations_list.append({
                "id": aff.get("@id"),
                "name": aff.get("affilname"),
                "city": aff.get("affiliation-city"),
                "country": aff.get("affiliation-country"),
            })

    # Research
    return {
        "id": data.get("coredata").get("dc:identifier").split(":")[1],
        "doi": data.get("coredata").get("prism:doi"),
        "eid": data.get("coredata").get("eid"),
        "cover_date": data.get("coredata").get("prism:coverDate"),
        "title": data.get("item").get("bibrecord").get("head").get("citation-title"),
        "abstract": data.get("item").get("bibrecord").get("head").get("abstracts"),
        "subject_areas": "|".join(set([field.get("@abbrev") for field in data.get("subject-areas").get("subject-area")])) if data.get("subject-areas").get("subject-area") else "",
        "auth_keywords": "|".join(set([field.get("$") for field in data.get("auth-keywords")])) if data.get("auth-keywords") else "",
        "authors_id": "|".join(set([field.get("@auid") for field in data.get("authors").get("author")])) if data.get("authors").get("author") else "",
        "citedby_count": data.get("coredata").get("citedby-count"),
        "ref_count": data.get("item").get("bibrecord").get("tail").get("bibliography").get("@refcount") if data.get("item").get("bibrecord").get("tail") else 0,
        "ref_ids": "|".join(new_ref_id_list),
        "published_year": data.get("coredata").get("prism:coverDate").split("-")[0],
        "published_month": data.get("coredata").get("prism:coverDate").split("-")[1],
        "published_day": data.get("coredata").get("prism:coverDate").split("-")[2],
    }
    
    

In [16]:
papers_list = []

for year in range(2018, 2024):
    path = f"./provided_data/Project/{year}"
    files = [f for f in listdir(path) if isfile(join(path, f))]

    for file in tqdm(files):
        with open(join(path, file), 'r', encoding="utf-8") as f:
            data = json.loads(f.read())
            json_obj = extract_json_from_data(data)
            papers_list.append(json_obj)

100%|██████████████████████████████████████████████████████████████████████████████| 2890/2890 [00:33<00:00, 87.06it/s]


## Researches

In [17]:
research_df = pd.json_normalize(papers_list) 
research_df.head()

,id,doi,eid,cover_date,title,abstract,subject_areas,auth_keywords,authors_id,citedby_count,ref_count,ref_ids,published_year,published_month,published_day
0,85077976956,10.1007/978-3-319-98485-8_15,2-s2.0-85077976956,2018-12-31,Public health and international epidemiology f...,None,MEDI,,36729660500|14720203700,1,76,0002667983|33750367977|85013970385|77953026614...,2018,12,31
1,85060936020,10.23919/PIERS.2018.8597669,2-s2.0-85060936020,2018-12-31,Flexible Printed Active Antenna for Digital Te...,"© 2018 The Institute of Electronics, Informati...",MATE|ENGI,,6507497381|57192376216,1,4,85006043726|85046336244|85060914424|85046368249,2018,12,31
2,85052201238,10.1016/j.ces.2018.08.042,2-s2.0-85052201238,2018-12-31,Parametric study of hydrogen production via so...,© 2018 Elsevier LtdComputational fluid dynamic...,CENG|CHEM|ENGI,,57202924002|50662017700|25923304100|7004487886,21,42,2942655685|84908055658|85052199786|84859716773...,2018,12,31
3,85051498032,10.1016/j.apsusc.2018.08.059,2-s2.0-85051498032,2018-12-31,Superhydrophobic coating from fluoroalkylsilan...,© 2018 Elsevier B.V. A superhydrophobic/supero...,MATE|CHEM|PHYS,,26436343700|57190429582|24074703800|5720338558...,37,45,78349312523|53249093621|84980335769|8486252720...,2018,12,31
4,85050678366,10.1016/j.aca.2018.07.045,2-s2.0-85050678366,2018-12-31,Electrochemical impedance-based DNA sensor usi...,© 2018 Elsevier B.V. A label-free electrochemi...,BIOC|ENVI|CHEM,,56524669400|6602082849|35364337500|8532633300|...,68,55,56249149272|33749077283|0037834610|73949151264...,2018,12,31


In [18]:
research_df.shape

(20216, 15)

## Authors

In [19]:
author_df = pd.json_normalize(authors_list)
author_df.head()

,id,given_name,initials,surname,indexed_name,affiliations_id
0,14720203700,Krit,K.,Pongpirul,Pongpirul K.,60006183|60028190|60020543
1,36729660500,Matthew P.,M.P.,Lungren,Lungren M.P.,60032838|60015615|60012708
2,57192376216,Teerapong,T.,Pratumsiri,Pratumsiri T.,60028190
3,6507497381,Panuwat,P.,Janpugdee,Janpugdee P.,60028190
4,57202924002,Kiattikhoon,K.,Phuakpunk,Phuakpunk K.,60028190


In [20]:
author_df.shape

(74257, 6)

## Affiliations

In [21]:
affiliation_df = pd.json_normalize(affiliations_list)
affiliation_df.head()

,id,name,city,country
0,60032838,Stanford University School of Medicine,Stanford,United States
1,60028190,Chulalongkorn University,Bangkok,Thailand
2,60020543,Bumrungrad International Hospital,Bangkok,Thailand
3,60015615,Stanford Healthcare,Stanford,United States
4,60012708,Stanford University,Palo Alto,United States


In [22]:
affiliation_df.shape

(12119, 4)

## Exporting to CSV

In [23]:
research_df.to_csv("researches.csv")
author_df.to_csv("authors.csv")
affiliation_df.to_csv("affiliations.csv")